This Notebook provides a minimal example for using LFP to train a simple LeNet on MNIST.

For more complex examples, refer to the experiment notebooks in ./nbs

### Imports

In [1]:
import os
import joblib
import random

import numpy as np
import torch
import torch.nn as tnn
import torcheval.metrics
import torchvision.datasets as tvisiondata
import torchvision.transforms as T
from tqdm import tqdm

from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from lfprop.propagation import (
    propagator_lxt as propagator,
)  # LFP propagator. Alternatively, use propagator_zennit
from lfprop.rewards import reward_functions as rewards  # Reward Functions
from lfprop.rewards import rewards as loss_fns

/home/lweber/.cache/pypoetry/virtualenvs/lfprop-KukTaqIE-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Parameters

In [2]:
model_name = "mlp" #lenet
optimizer_name = "sgd"
savepath = f"/media/lweber/f3ed2aae-a7bf-4a55-b50d-ea8fb534f1f52/reward-backprop/resubmission-1-experiments/test-regression/{model_name}-{optimizer_name}"
os.makedirs(savepath, exist_ok=True)

input_size = 8
lr = 0.01
batch_size = 128
epochs = 100
momentum = 0.9

seed = 0

def set_random_seeds(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.enabled = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Load Dataset

In [3]:
class PrepareData(torch.utils.data.Dataset):

    def __init__(self, X, y, scale_X=True):
        if not torch.is_tensor(X):
            if scale_X:
                X = StandardScaler().fit_transform(X)
                self.X = torch.from_numpy(X)
        if not torch.is_tensor(y):
            self.y = torch.from_numpy(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx].type(torch.float32), self.y[idx].type(torch.float32)

X, y = fetch_california_housing(return_X_y=True)

# create train and test indices
train, test = train_test_split(list(range(X.shape[0])), test_size=.3)

ds = PrepareData(X, y=y, scale_X=True)

training_loader = torch.utils.data.DataLoader(ds, batch_size=batch_size,
                       sampler=torch.utils.data.SubsetRandomSampler(train))
test_loader = torch.utils.data.DataLoader(ds, batch_size=batch_size,
                      sampler=torch.utils.data.SubsetRandomSampler(test))

### Load Model

In [4]:
class RegressionModel(torch.nn.Module):

    def __init__(self, input_size, activation=tnn.ReLU):
        super(RegressionModel, self).__init__()
        self.features = tnn.Sequential(
            torch.nn.Linear(in_features=input_size, out_features=256),
            activation(),
            torch.nn.Dropout(p=0.5),
            torch.nn.Linear(in_features=256, out_features=128),
            activation(),
            torch.nn.Dropout(p=0.5),
            torch.nn.Linear(in_features=128, out_features=1)
        )

    def forward(self, X):

        return self.features(X)

model = RegressionModel(input_size=input_size)
model.to(device)

RegressionModel(
  (features): Sequential(
    (0): Linear(in_features=8, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=128, out_features=1, bias=True)
  )
)

### Set Up LFP

In [5]:
# Initialize the LFP Composite (cf. "composites" in zennit or lxt).
# This call is the same whether the lxt or zennit backend is used (propagator_lxt and propagator_zennit).
# Currently, only LFP-Epsilon is implemented. More composites may be added in the future.
propagation_composite = propagator.LFPEpsilonComposite()
#propagation_composite = propagator.LFPHebbianEpsilonComposite(use_oja=True)
#propagation_composite = propagator.LFPGammaComposite(gamma=0.0)

# Initialize the Reward Function.
# Here we use the Reward Function suggested in the LFP-Paper, but check out other reward functions in ./lfp/rewards/reward_functions.py
class RegressionReward:
    def __init__(self, device, **kwargs):
        """
        Computes regression reward
        """
        self.device = device

    def __call__(self, logits, labels):
        """
        Computation
        :param logits:
        :param labels:
        :return:
        """

        # Compute reward
        reward = (labels.view(logits.shape) - logits)**3 * logits.sign()
        return reward
    
reward_func = RegressionReward(device)

# LFP writes its updates into the .grad attribute of the model parameters, and can thus utilize standard torch optimizers
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

### Set Up Simple Evaluation using torcheval

In [6]:
def eval_model(loader):
    """
    Evaluates the model on a single dataset
    """
    eval_metrics = {
        "reward": torcheval.metrics.Mean(device=device),
        "mse": torcheval.metrics.MeanSquaredError(device=device),
    }

    model.eval()

    # Iterate over Data Loader
    for index, (inputs, labels) in enumerate(loader):
        inputs = inputs.to(device)
        labels = torch.tensor(labels).to(device)

        with torch.no_grad():
            # Get model predictions
            outputs = model(inputs)

        with torch.set_grad_enabled(True):
            # Get rewards
            reward = reward_func(outputs, labels)

        for k, v in eval_metrics.items():
            if k == "reward":
                eval_metrics[k].update(reward)
            else:
                eval_metrics[k].update(outputs.view(labels.shape), labels)

    return_dict = {m: metric.compute().detach().cpu().numpy() for m, metric in eval_metrics.items()}

    # Return evaluation
    return return_dict

### Training Loop

In [7]:
def lfp_step(inputs, labels):
    """
    Performs a single training step using LFP. This is quite similar to a standard gradient descent training loop.
    """
    # Set Model to training mode
    model.train()

    with torch.enable_grad():
        # Zero Optimizer
        optimizer.zero_grad()

        # This applies LFP Hooks/Functions (which depends on whether lxt or zennit backend is used)
        with propagation_composite.context(model) as modified:
            inputs = inputs.detach().requires_grad_(True)
            outputs = modified(inputs)

            # Calculate reward
            # Do like this to avoid tensors being kept in memory
            reward = torch.from_numpy(reward_func(outputs, labels).detach().cpu().numpy()).to(device)

            # Calculate LFP and write into .feedback attribute of parameters
            torch.autograd.grad((outputs,), (inputs,), grad_outputs=(reward,), retain_graph=False)[0]

            # Write LFP Values into .grad attributes. Note the negative sign: LFP requires maximization instead of minimization like gradient descent
            for name, param in model.named_parameters():
                param.grad = -param.feedback

            # Update Clipping. Training may become unstable otherwise, especially in small models with large learning rates.
            # In larger models (e.g., VGG, ResNet), where smaller learning rates are generally utilized, not clipping updates may result in better performance.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 3.0, 2.0)

            # Optimization step
            optimizer.step()

    # Set Model back to eval mode
    model.eval()


# Training Loop
result = {
    "mse_train": [],
    "mse_test": [],
    "reward_train": [],
    "reward_test": [],
}
for epoch in range(epochs):
    with tqdm(total=len(training_loader)) as pbar:
        # Iterate over Data Loader
        for index, (inputs, labels) in enumerate(training_loader):
            inputs = inputs.to(device)
            labels = torch.tensor(labels).to(device)

            # Perform Update Step
            lfp_step(inputs, labels)

            # Update Progress Bar
            pbar.update(1)

    # Evaluate and print performance after every epoch
    eval_stats_train = eval_model(training_loader)
    eval_stats_test = eval_model(test_loader)
    print(
        "Epoch {}/{}: (Train Reward) {:.2f}; (Train MSE) {:.2f}; (Val Reward) {:.2f}; (Val MSE) {:.2f}".format(
            epoch + 1,
            epochs,
            float(np.mean(eval_stats_train["reward"])),
            float(eval_stats_train["mse"]),
            float(np.mean(eval_stats_test["reward"])),
            float(eval_stats_test["mse"]),
        )
    )

    result["mse_train"].append(float(np.mean(eval_stats_train["mse"])))
    result["mse_test"].append(float(np.mean(eval_stats_test["mse"])))
    result["reward_train"].append(float(np.mean(eval_stats_train["reward"])))
    result["reward_test"].append(float(np.mean(eval_stats_test["reward"])))

  0%|          | 0/113 [00:00<?, ?it/s]/tmp/ipykernel_439267/3515831022.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).to(device)
/home/lweber/.cache/pypoetry/virtualenvs/lfprop-KukTaqIE-py3.11/lib/python3.11/site-packages/lxt/core.py:362: UserWarning: This functionality is not yet fully tested. Please check the model after removing the composite.
  warn(
100%|██████████| 113/113 [00:01<00:00, 72.84it/s]
/tmp/ipykernel_439267/2800525172.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).to(device)


Epoch 1/100: (Train Reward) -0.01; (Train MSE) 0.65; (Val Reward) 0.04; (Val MSE) 0.63


100%|██████████| 113/113 [00:00<00:00, 119.35it/s]


Epoch 2/100: (Train Reward) 0.09; (Train MSE) 0.61; (Val Reward) 0.10; (Val MSE) 0.60


100%|██████████| 113/113 [00:00<00:00, 118.85it/s]


...

Epoch 98/100: (Train Reward) 0.08; (Train MSE) 0.33; (Val Reward) 0.07; (Val MSE) 0.34


100%|██████████| 113/113 [00:00<00:00, 118.60it/s]


Epoch 99/100: (Train Reward) -0.31; (Train MSE) 0.49; (Val Reward) -0.33; (Val MSE) 0.51


100%|██████████| 113/113 [00:00<00:00, 120.02it/s]


Epoch 100/100: (Train Reward) 0.01; (Train MSE) 0.34; (Val Reward) -0.00; (Val MSE) 0.35
